# CSQA BGE-small search

In [1]:
import pandas as pd
import numpy as np
from transformers import AutoModel, AutoTokenizer
import faiss
import torch
from tqdm.auto import trange
import pickle
import re
import torch.functional as F

In [2]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(x):
    """Функция, создающая запрос для bge"""
    return f"{x['question_state']} {x['question']}"

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = -100
    return df

In [3]:
# Загружаем данные
train_data = make_df('csqa_data/train.statement.jsonl')
vall_data = make_df('csqa_data/dev.statement.jsonl')
test_data = make_df('csqa_data/test.statement.jsonl',is_test=True)

In [4]:
# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [5]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
def cls_pooling(model_output):
    return model_output[0][:, 0]

@torch.no_grad()
def create_embeds(texts,model,tokenizer,batch_size=64,max_length=1024):
    """Функция, для получения векторных представлений из текста"""
    features = []
    texts = [prepare_text(x) for x in texts]
    for i in trange(len(texts) // batch_size + 1):
        encs = tokenizer(
            texts[i*batch_size:(i+1)*batch_size],
            truncation=True,
            padding='longest',
            max_length=max_length,
            return_tensors='pt'
        )
        encs['input_ids'] = encs['input_ids'].cuda()
        encs['attention_mask'] = encs['attention_mask'].cuda()
        if 'token_type_ids' in encs.keys():
            encs['token_type_ids'] = encs['token_type_ids'].cuda()
        model_output = model(**encs)
        embeds = cls_pooling(model_output)#mean_pooling(model_output, encs['attention_mask'])
        embeds = torch.nn.functional.normalize(embeds, p=2, dim=1).detach().cpu().numpy()
        features.append(embeds)
    return np.concatenate(features,dtype=np.float32)

In [6]:
encoder = AutoModel.from_pretrained('BAAI/bge-small-en-v1.5').cuda()
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-small-en-v1.5')

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

2024-10-24 12:06:43.157748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 12:06:43.157800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 12:06:43.158737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 12:06:43.164554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 12:06:43.966294: W tensorflow/compiler/tf2

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [8]:
# делаем препроцессинг вершин
concepts = [prepare_text(s)  if type(s) == str else ' ' for s in graph_state['concepts_maper']]

In [9]:
features = create_embeds(concepts,encoder,tokenizer,batch_size=512)

  0%|          | 0/1562 [00:00<?, ?it/s]

In [10]:
with open('faiss_embeds.pkl','wb') as f:
    pickle.dump(features,f)

In [11]:
# Создаём faiss index для быстрого поиска кандидатов
index = faiss.IndexFlatIP(features.shape[1])
faiss.normalize_L2(features)
index.add(features)
res = faiss.StandardGpuResources()
index = faiss.index_cpu_to_gpu(res, 0, index)
index.ntotal

799272

In [12]:
# Векторизуем запросы
train_features = create_embeds(train_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
test_features = create_embeds(test_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)
val_features = create_embeds(vall_data['request'].tolist(),encoder,tokenizer,batch_size=128,max_length=512)

  0%|          | 0/62 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
# Ищем по запросам релевантные вершины 
faiss.normalize_L2(train_features)
D_train,I_train = index.search(train_features,k=7)
faiss.normalize_L2(test_features)
D_test,I_test = index.search(test_features,k=7)
faiss.normalize_L2(val_features)
D_vall,I_vall = index.search(val_features,k=7)

In [14]:
state_bge = {
    'train_cnpt': I_train,
    'test_cpnt': I_test,
    'vall_cnpt': I_vall,
}

with open('./csqa_data/bge_ids.pkl','wb') as f:
    pickle.dump(state_bge,f)

Если у вас мало RAM, то рекомендуется перезагрузить среду

# Lemmas Search

In [18]:
import numpy as np
import pandas as pd
import re
import multiprocessing as mp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import spacy
import pickle
from string import punctuation
from spacy.lang.en import stop_words

2024-10-24 12:08:43.648642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-24 12:08:43.648984: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-24 12:08:43.649074: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [19]:
def prepare_text(s):
    """Функция предобработки текста"""
    return re.sub(" +", " ",s.lower().replace('_',' ').strip())

def parse_request(x):
    """Функция, создающая запрос для bge"""
    return f"{x['question_state']} {x['question']}"

def make_df(path,is_test=False):
    """"Предобработка данных"""
    data = pd.read_json(path,lines=True)
    df = pd.DataFrame()
    
    df['question_state'] = data['question'].apply(lambda x: x['question_concept'])
    df['id'] = data['id']
    df['question'] = data['question'].apply(lambda x: x['stem'])
    df['choice_A'] = data['question'].apply(lambda x: x['choices'][0]['text'])
    df['choice_B'] = data['question'].apply(lambda x: x['choices'][1]['text'])
    df['choice_C'] = data['question'].apply(lambda x: x['choices'][2]['text'])
    df['choice_D'] = data['question'].apply(lambda x: x['choices'][3]['text'])
    df['choice_E'] = data['question'].apply(lambda x: x['choices'][4]['text'])
    
    if not is_test:
        df['label'] = data['answerKey'].map({'A':0,'B':1,'C':2,'D':3,'E':4})
    else:
        df['label'] = -100
    return df

In [20]:
# Загружаем данные
train_data = make_df('csqa_data/train.statement.jsonl')
vall_data = make_df('csqa_data/dev.statement.jsonl')
test_data = make_df('csqa_data/test.statement.jsonl',is_test=True)

In [21]:
# Сопоставляем запросы
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [22]:
class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        """Базовая предобработка текста перед лемматизацией"""
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        """Лемматизируем запрос и удаляем стоп слова"""
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

In [23]:
class EntityLinlker():
    def __init__(self, kg, text_proc, prepared2concept):
        self.kg = kg
        self.text_proc = text_proc
        self.p2c = prepared2concept
        
    def link_sentence(self,s):
        """Ищем для запроса релевантных кандидатов"""
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        """Для увеличения скорости паралелим вычесления"""
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts

In [24]:
processor = TextProcessor()
graph_state = pickle.load(open('graph.pkl','rb'))
concepts = graph_state['concepts_maper'].keys()
concepts = [(x if type(x) == str else '') for x in concepts] #delete valid nodes

In [25]:
prepared_concepts = processor.process_batch(concepts)# pickle.load(open('conceprs_enc.pkl','rb')) - если не хотите ждать
concept2prepared = dict(zip(concepts,prepared_concepts)) # словарь, сопоставляющий концепт -> его лемматезированная версия

  0%|          | 0/799272 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [41]:
def texual_acc(s1,s2): # like charackter bleu@1 score
    return sum(i == j for i,j in zip(s1,s2)) / (max(len(s1),len(s2)) + 1e-6)

def prep_text(s):
    return s.lower().strip().replace('_', ' ')

def rank_func(s1,texts):
    s1 = prep_text(s1)
    scores = []
    for s in texts:
        scores.append(texual_acc(s1,prep_text(s)))
    return texts[np.argmax(scores)]

In [27]:
# из за лемматизации, текстовая составляющая некоторых вершин стала идентичной
# по этому для их текста ищем вершину, чья лемматезированная версия как можно сильнее совподает с не лемматезированной
# таким образом получаем словарь, который сопоставляет лемматизированному концепту его исходную версию
df_cp = pd.DataFrame(zip(concepts,prepared_concepts))
df_cp[1] = df_cp[1].apply(lambda x: ' '.join(x))
df_cp_grp = df_cp.groupby(1)[0].agg(list)
ranked_nodes = [rank_func(y,x) for x,y in tzip(df_cp_grp.tolist(), df_cp_grp.index.tolist())]
prepared2concept = dict(zip(df_cp_grp.index.tolist(),ranked_nodes))

  0%|          | 0/656261 [00:00<?, ?it/s]

In [28]:
with open('prepared2concept.pkl','wb') as f:
    pickle.dump(prepared2concept,f)

In [29]:
linker = EntityLinlker(prepared_concepts, processor, prepared2concept)

In [30]:
# ищем кандидатов
train_data['candidates'] = linker.batch_linking(train_data['request'].tolist())
vall_data['candidates'] = linker.batch_linking(vall_data['request'].tolist())
test_data['candidates'] = linker.batch_linking(test_data['request'].tolist())

  0%|          | 0/7893 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/877 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/2192 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [31]:
def get_concepts(ids):
    """Преобразуем концепты в число"""
    ids = [prepared_concepts[i] for i in ids]
    ids = [' '.join(x) for x in ids]
    return [graph_state['concepts_maper'][prepared2concept[i]] for i in ids] #[concepts.index(prepared2concept[i]) for i in ids] 

In [32]:
test_data['candidates'] = test_data['candidates'].apply(get_concepts)
train_data['candidates'] = train_data['candidates'].apply(get_concepts)
vall_data['candidates'] = vall_data['candidates'].apply(get_concepts)

In [33]:
idf_matrix = pd.Series(
    np.concatenate(test_data['candidates'].tolist() + train_data['candidates'].tolist() + vall_data['candidates'].tolist())
).value_counts()
idf_matrix = idf_matrix.to_dict()

In [34]:
with open('idf_csqa.pkl','wb') as f:
    pickle.dump(idf_matrix,f)

In [35]:
# Берём семплы с наименьшей популярностью, так как они будут более значимыми для нашей модели
# Таким образом получается получить top-k самых редких семплов, о которых исходная LM скорее всего не обладает знаниями
def rank_by_idf(texts,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [idf_matrix[x] for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

In [36]:
test_data['candidates_rank'] = test_data['candidates'].apply(rank_by_idf)
train_data['candidates_rank'] = train_data['candidates'].apply(rank_by_idf)
vall_data['candidates_rank'] = vall_data['candidates'].apply(rank_by_idf)

In [37]:
test_data['candidates_rank']

0        [37853, 65505, 11257, 107123, 2979]
1           [13504, 353, 11261, 9946, 11204]
2             [3425, 3709, 1380, 2239, 2980]
3       [20906, 572340, 214884, 10205, 6708]
4                         [19377, 538, 2571]
                        ...                 
2187    [367671, 410026, 65592, 16498, 3300]
2188      [229088, 15982, 13671, 581, 27632]
2189                            [4218, 2735]
2190              [5481, 63005, 2106, 70991]
2191                           [17799, 2735]
Name: candidates_rank, Length: 2192, dtype: object

# Merge All candidates

In [38]:
bge_state = pickle.load(open('./csqa_data/bge_ids.pkl','rb'))

In [39]:
def match_pairs(c1,c2):
    "Матчим кандидатов"
    state = c1+c2.tolist()
    idx = np.unique(state, return_index=True)[1]
    return [graph_state['concepts_maper_inv'][state[i]] for i in sorted(idx)][::-1]

state = {
    'train_all_cands': [match_pairs(c1,c2) for c1,c2 in zip(train_data['candidates_rank'].tolist(),bge_state['train_cnpt'])],
    'test_all_cands': [match_pairs(c1,c2) for c1,c2 in zip(test_data['candidates_rank'].tolist(),bge_state['test_cpnt'])],
    'vall_all_cands': [match_pairs(c1,c2) for c1,c2 in zip(vall_data['candidates_rank'].tolist(),bge_state['vall_cnpt'])],
}

In [40]:
state['train_all_cands'][239]

['contraceptive_diaphragm',
 'contraceptive_device',
 'condomn',
 'pessary',
 'prescription_drug_market_category',
 'avoid_pregnancy',
 'contraceptive']

In [23]:
with open('./csqa_data/all_candsV3.pkl','wb') as f:
    pickle.dump(state, f)